In [188]:
import numpy as np
import pandas as pd
import googlemaps
from os import listdir
from os.path import isfile, join

In [189]:
df = pd.read_csv('TEP Data/PassengerActivity_ByTimePeriod_RT_22.csv')
df_ob = df.loc[1:52].reset_index(drop=True)
df_ib = df.loc[57:101].reset_index(drop=True)

In [190]:
#Added people-seconds
4*200/1.4

571.4285714285714

In [191]:
#Saved people seconds
291*10

2910

In [192]:
df_ob['Daily On'] = df_ob['Daily On'].astype(int)
df_ob['Daily Off'] = df_ob['Daily Off'].astype(int)
df_ob['Daily Load'] = df_ob['Daily Load'].astype(int)

In [193]:
df_ob['time_added'] = df_ob['Daily On']*200/1.4
df_ob['time_saved'] = (df_ob['Daily Load'] - df_ob['Daily On'])*10
df_ob['time_delta'] = df_ob['time_added'] - df_ob['time_saved']

In [194]:
df_ob['distance_to_next_stop'] = 200
df_ob['prev_stop_idx'] = df_ob.index-1
df_ob['next_stop_idx'] = df_ob.index+1
df_ob['removed_flag'] = False

In [201]:
df_ob

,Stop,Daily On,Daily Off,Daily Load,AM Peak On,AM Peak Off,AM Peak Load,Midday On,Midday Off,Midday Load,...,Owl On,Owl Off,Owl Load,time_added,time_saved,time_delta,distance_to_next_stop,prev_stop_idx,next_stop_idx,removed_flag
0,Fillmore St&Bay St SE-NS/BZ,267,0,291,29,0,36,76,0,82,...,24,0,27,38142.857143,240,37902.857143,200,-1,1,False
1,Fillmore St&North Point St SE-NS,4,0,295,0,0,36,3,0,85,...,0,0,27,571.428571,2910,-2338.571429,200,0,2,False
2,Fillmore St&Beach St SE-NS/BZ,9,0,304,1,0,37,7,0,92,...,0,0,27,1285.714286,2950,-1664.285714,200,1,3,False
3,Fillmore St&Jefferson St SW-FS/BZ,37,17,324,17,2,52,11,9,94,...,1,0,28,5285.714286,2870,2415.714286,200,2,4,False
4,Fillmore St&Beach St NW-NS/BZ,108,9,423,27,0,79,23,2,115,...,3,0,31,15428.571429,3150,12278.571429,200,3,5,False
5,Fillmore St&Cervantes Blvd NW,85,1,507,19,0,98,28,0,143,...,0,0,31,12142.857143,4220,7922.857143,200,4,6,False
6,Fillmore St&Chestnut St NW-NS/BZ,506,41,972,47,15,130,99,9,233,...,49,3,77,72285.714286,4660,67625.714286,200,5,7,False
7,Fillmore St&Lombard St SW-FS/BZ,394,22,1344,59,2,187,91,4,320,...,27,1,103,56285.714286,9500,46785.714286,200,6,8,False
8,Union St&Fillmore St NW-FS/BZ,383,71,1656,32,13,206,85,20,385,...,24,0,127,54714.285714,12730,41984.285714,200,7,9,False
9,Steiner St&Union St SW-FS/BZ,97,8,1748,13,0,220,24,3,408,...,7,0,134,13857.142857,16510,-2652.857143,200,8,10,False


In [237]:
#df_new = df_ob.copy()

#while df_new['time_delta'].min() < 0:
idx = df_new[df_new['removed_flag'] == False]['time_delta'].idxmin()

stop = df_new.iloc[idx]['Stop']
prev_stop_idx = df_new.iloc[idx]['prev_stop_idx']
next_stop_idx = df_new.iloc[idx]['next_stop_idx']

daily_on = df_new.iloc[idx]['Daily On']
daily_off = df_new.iloc[idx]['Daily Off']
extra_distance = df_new.iloc[idx]['distance_to_next_stop']

df_new.at[next_stop_idx, 'Daily On'] = df_new.iloc[next_stop_idx]['Daily On'] + daily_on
df_new.at[prev_stop_idx, 'Daily Off'] = df_new.iloc[prev_stop_idx]['Daily Off'] + daily_off
df_new.at[prev_stop_idx, 'Daily Load'] = df_new.iloc[prev_stop_idx]['Daily Load'] - daily_off
df_new.at[prev_stop_idx, 'distance_to_next_stop'] = df_new.iloc[prev_stop_idx]['distance_to_next_stop'] + extra_distance

df_new.at[prev_stop_idx, 'next_stop_idx'] = next_stop_idx
df_new.at[next_stop_idx, 'prev_stop_idx'] = prev_stop_idx

#df_new.at[idx, 'next_stop_idx'] = None
#df_new.at[idx, 'prev_stop_idx'] = None

df_new.at[idx, 'removed_flag'] = True

print(stop)
print(df_new[df_new['removed_flag'] == False]['time_delta'].min())
print(df_new)


IndexError: single positional indexer is out-of-bounds

In [241]:
df_new.to_csv('optimized_22.csv')

In [243]:
df_new[df_new['removed_flag'] == False]

,Stop,Daily On,Daily Off,Daily Load,AM Peak On,AM Peak Off,AM Peak Load,Midday On,Midday Off,Midday Load,...,Owl On,Owl Off,Owl Load,time_added,time_saved,time_delta,distance_to_next_stop,prev_stop_idx,next_stop_idx,removed_flag
0,Fillmore St&Bay St SE-NS/BZ,267,0,291,29,0,36,76,0,82,...,24,0,27,38142.857143,240,37902.857143,600,-1,3,False
3,Fillmore St&Jefferson St SW-FS/BZ,50,17,324,17,2,52,11,9,94,...,1,0,28,5285.714286,2870,2415.714286,200,0,4,False
4,Fillmore St&Beach St NW-NS/BZ,108,9,423,27,0,79,23,2,115,...,3,0,31,15428.571429,3150,12278.571429,200,3,5,False
5,Fillmore St&Cervantes Blvd NW,85,1,507,19,0,98,28,0,143,...,0,0,31,12142.857143,4220,7922.857143,200,4,6,False
6,Fillmore St&Chestnut St NW-NS/BZ,506,41,972,47,15,130,99,9,233,...,49,3,77,72285.714286,4660,67625.714286,200,5,7,False
7,Fillmore St&Lombard St SW-FS/BZ,394,22,1344,59,2,187,91,4,320,...,27,1,103,56285.714286,9500,46785.714286,200,6,8,False
8,Union St&Fillmore St NW-FS/BZ,383,150,1577,32,13,206,85,20,385,...,24,0,127,54714.285714,12730,41984.285714,1200,7,14,False
14,Jackson St&Fillmore St NW-FS/BZ,217,0,2,NaN,NaN,NaN,0,0,0,...,1,0,1,285.714286,0,285.714286,200,8,15,False
15,Fillmore St&Jackson St SW-FS/BZ,166,160,1803,30,30,219,41,38,419,...,5,4,137,23714.285714,16370,7344.285714,200,14,16,False
16,Fillmore St&Sacramento St NW,513,244,2072,40,43,216,175,75,519,...,22,10,149,73285.714286,15590,57695.714286,200,15,17,False


In [45]:
'''gmaps = googlemaps.Client(key=gmaps_api_key)
def get_lat_lng(row):
    
    address = row['State']
    
    if isinstance(row['City'], str):
        address = row['City'] + ', ' + address
        
    if isinstance(row['Address'], str):
        address = row['Address'] + ', ' + address

    try:
        geocode_result = gmaps.geocode(address)
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        print (address + ' is at ' + str(lat) + ', ' + str(lng))
        
    except Exception as e:
        print(address)
        print(e)
        lat = np.NaN
        lng = np.NaN
        
    return lat, lng'''

"gmaps = googlemaps.Client(key=gmaps_api_key)\ndef get_lat_lng(row):\n    \n    address = row['State']\n    \n    if isinstance(row['City'], str):\n        address = row['City'] + ', ' + address\n        \n    if isinstance(row['Address'], str):\n        address = row['Address'] + ', ' + address\n\n    try:\n        geocode_result = gmaps.geocode(address)\n        lat = geocode_result[0]['geometry']['location']['lat']\n        lng = geocode_result[0]['geometry']['location']['lng']\n        print (address + ' is at ' + str(lat) + ', ' + str(lng))\n        \n    except Exception as e:\n        print(address)\n        print(e)\n        lat = np.NaN\n        lng = np.NaN\n        \n    return lat, lng"

In [46]:
'''def load_gtfs_data(path='google_transit'):
    #Returns dictionary of dataframes containing GTFS data
    files = [f for f in listdir(path) if isfile(join(path, f))]
    df = {}
    for file in files:
        print("Loading {}".format(file[:-4]))
        try:
            df[file[:-4]] = pd.read_csv(path + '/' + file)
        except:
            print("{} failed to load!".format(file[:-4]))
            continue
    return df
'''

'def load_gtfs_data(path=\'google_transit\'):\n    #Returns dictionary of dataframes containing GTFS data\n    files = [f for f in listdir(path) if isfile(join(path, f))]\n    df = {}\n    for file in files:\n        print("Loading {}".format(file[:-4]))\n        try:\n            df[file[:-4]] = pd.read_csv(path + \'/\' + file)\n        except:\n            print("{} failed to load!".format(file[:-4]))\n            continue\n    return df\n'

In [47]:
'''gtfs_dict = load_gtfs_data()'''

'gtfs_dict = load_gtfs_data()'